## Install Prerequisites

In [13]:
import subprocess
import sys
import base64
import getpass
import requests
import yaml
import matplotlib
import plotly.graph_objects as go
import networkx as nx
import webbrowser

To be able to show the whigets run:
```
pip install --upgrade pip
pip install ipywidgets
jupyter nbextension enable --py widgetsnbextension
jupyter labextension install @jupyter-widgets/jupyterlab-manager
jupyter lab build
```
Close the Jupyter lab window and open it again 

# Client Configuration

In [15]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import TwoByTwoLayout
from ipywidgets import AppLayout, Button, Layout


sdia_conf_url_widget =  widgets.Text(
    value='https://lifewatch.lab.uvalight.net:30736/orchestrator',
    placeholder='Enter SDIA API endpoint. e.g. https://lifewatch.lab.uvalight.net:30736/orchestrator',
    description='SDIA API URL:',
    disabled=False
)
sdia_conf_username_widget =  widgets.Text(
    value='notebook_user',
    placeholder='Enter SDIA username',
    description='SDIA username:',
    disabled=False
)

sdia_conf_token_widget =  widgets.Password(
    value='',
    placeholder='Enter your SDIA token',
    description='Password:',
    disabled=False
) 

sdia_conf_login_button = widgets.Button(
    description='Login',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Run report',
#     icon='check' # (FontAwesome names without the `fa-` prefix)
)

sdia_conf_output = widgets.Output()
sdia_conf_box = widgets.VBox([sdia_conf_url_widget, sdia_conf_username_widget, sdia_conf_token_widget, sdia_conf_login_button])
display(sdia_conf_box,sdia_conf_output)

# @sdia_conf_output.capture()
# def sdia_conf_on_button_clicked(b):
#     sdia_conf_url = sdia_conf_url_widget.value
# #     if not sdia_token_widget.value or not sdia_username_widget.value:
        

# sdia_conf_login_button.on_click(sdia_conf_on_button_clicked)



Output()

# Application Configuration

In [17]:
application_type_widget = widgets.Dropdown(
    options=['Docker', 'Helm'],
    value='Helm',
    description='Application Type:',
    disabled=False
)

application_type_output = widgets.Output()
application_type_box = widgets.VBox([application_type_widget])
display(application_type_box,application_type_output)

Output()

In [33]:
docker_app_image_name_widget =  widgets.Textarea(
    value='cloudcells/classifiers',
    placeholder='image name e.g. cloudcells/classifiers',
    description='Docker Image Name:',
    disabled=False
)

docker_app_image_ports_widget =  widgets.Textarea(
    value='',
    placeholder='the ports to expose e.g. 80',
    description='Ports:',
    disabled=False
)

enebale_monotoring_widget= widgets.Checkbox(
    value=True,
    description='Monotoring',
    disabled=False
)

app_name_widget =  widgets.Text(
    value='app',
    placeholder='the applications name',
    description='Application Name:',
    disabled=False
)

docker_app_box = widgets.VBox([app_name_widget,docker_app_image_name_widget, docker_app_image_ports_widget,enebale_monotoring_widget])

                    
helm_app_chart_name_widget =  widgets.Text(
    value='argo/argo',
    placeholder='the chart name e.g. argo/argo',
    description='Chart Name:',
    disabled=False
)


helm_app_repo_name_widget =  widgets.Text(
    value='argo',
    placeholder='the name of the repoitory name',
    description='Reposetory Name:',
    disabled=False
)

helm_app_repo_url_widget =  widgets.Text(
    value='https://argoproj.github.io/argo-helm',
    placeholder='the url',
    description='Reposetory URL:',
    disabled=False
)

helm_app_box = widgets.VBox([app_name_widget,helm_app_chart_name_widget, helm_app_repo_name_widget,helm_app_repo_url_widget,enebale_monotoring_widget])

app_conf_output = widgets.Output()

if application_type_widget.value == 'Docker':
    display(docker_app_box,app_conf_output)
elif application_type_widget.value == 'Helm':
    display(helm_app_box,app_conf_output)


Output()

# Infrastructure Configuration

## Number of Topologies
A topology is a set of VMs that will be provisined in the same location by the same provider.

In [20]:
num_of_topologies_widget= widgets.BoundedIntText(
    value=1,
    min=1,
    max=4,
    step=1,
    description='Num. Of Topologies:',
    disabled=False
)


topologies_output = widgets.Output()
number_of_topology_box = widgets.VBox([num_of_topologies_widget])
display(number_of_topology_box,topologies_output)


Output()

In [49]:
topologies_boxes = []
region_names = ['East US','Central US','South Central US','West US','Australia East','Southeast Asia','UK South','West Europe','North Europe','South Africa North','Central India']


for i in range(0, num_of_topologies_widget.value):
    cloud_provider_name_widget = widgets.Dropdown(
        options=['EC2', 'Azure'],
        value='Azure',
        description='Cloud Provider:',
        name='cloud_provider_name',
        disabled=False
    )
    
    num_of_vms_widget= widgets.BoundedIntText(
        value=2,
        min=1,
        max=4,
        step=1,
        description='Num. Of VMs:',
        disabled=False
    )
    vm_size_name_widget = widgets.Dropdown(
        options=['small', 'medium','large'],
        value='medium',
        description='VM size:',
        disabled=False
    )
    topology_region_widget = widgets.Dropdown(
        options=region_names,
        value=region_names[7],
        description='Topology Region:',
        disabled=False
    )
    topology_box = widgets.VBox([cloud_provider_name_widget, num_of_vms_widget, vm_size_name_widget,topology_region_widget])
    topologies_boxes.append(topology_box)

    
    
topologies_conf_tab = widgets.Tab()
topologies_conf_tab.children = topologies_boxes
for i in range(len(topologies_boxes)):
    topologies_conf_tab.set_title(i, 'topology_'+str(i+1))
    
    
topologies_conf_output = widgets.Output()
topologies_conf_tab_box = widgets.VBox([topologies_conf_tab])
display(topologies_conf_tab_box,topologies_conf_output)

Output()

## Generate TOSCA

In [88]:
node_templates={}
app_prop = []

if application_type_widget.value == 'Docker':
    app_prop.append('Image Name: '+docker_app_image_name_widget.value)
    app_prop.append('Ports: '+docker_app_image_ports_widget.value)
elif application_type_widget.value == 'Helm':
    app_prop.append('Application Type: '+application_type_widget.value)
    app_prop.append('Chart name: '+helm_app_chart_name_widget.value)
    app_prop.append('Reposetory name: '+helm_app_repo_name_widget.value)
    app_prop.append('Reposetory URL: '+helm_app_repo_url_widget.value)

text = 'Application\n'
for line in app_prop:
    text += line+'\n'
    

def build_node_template(node_name=None,node_type=None,properties=None,requirements=None,interfaces=None):
    node_template = {node_name:{'properties':properties,'requirements':requirements,'interfaces':interfaces}}
    return node_template
    
def translate_region(cloud_provider,region_name):
    if cloud_provider == 'Azure':
        return region_name.lower().replace(' ','')
    
    
def translate_vm_size(cloud_provider,vm_size):
    vm_specs = {}
    if vm_size == 'small':
        print(vm_size)
        vm_specs['num_cores']='1'
        vm_specs['mem_size']='2048 MB'
        vm_specs['disk_size']='10000 MB'
    elif vm_size == 'medium':
        vm_specs['num_cores']='2'
        vm_specs['mem_size']='4048 MB'
        vm_specs['disk_size']='20000 MB'
    elif vm_size == 'large':
        vm_specs['num_cores']='4'
        vm_specs['mem_size']='8048 MB'
        vm_specs['disk_size']='40000 MB'
    return vm_specs
    
    
text += '\nInfrastructure\n'

index=0
topology_nodes = []
for topology in topologies_boxes:
    index+=1
    topology_wiget_inputs={}
    topology_wiget_inputs['name'] = 'topology_'+str(index)
    for child in topology.children:
        text+=child.description+':'+str(child.value)+'\n'
        topology_wiget_inputs[child.description.replace(':','').replace(' ','_')]=child.value    
        
        
    topology_properties={}
    topology_properties['region']=translate_region(topology_wiget_inputs['Cloud_Provider'],topology_wiget_inputs['Topology_Region'])
    topology_properties['provider'] = topology_wiget_inputs['Cloud_Provider']
    topology_requirements=[]
    for i in range(topology_wiget_inputs['Num._Of_VMs']):
        vm_name = 'compute_'+str(i)+'_'+topology_wiget_inputs['name']
        vm_properties = {'os_distro': 'Ubuntu','os_version':'18.04','user_name': 'vm_user'}
        vm_properties.update(translate_vm_size(topology_properties['provider'],topology_wiget_inputs['VM_size']))
        vm_interfaces={'interfaces':{'Standard':{'create':'dumy.yaml'}}}
        topology_nodes.append(build_node_template(node_name=vm_name,node_type='tosca.nodes.QC.VM.Compute',properties=vm_properties,interfaces=vm_interfaces))
        vm_req={'vm':{'capability': 'tosca.capabilities.QC.VM','node':vm_name ,'relationship':'tosca.relationships.DependsOn'}}
        topology_requirements.append(vm_req)
    topology_interfaces
    topology_nodes.append(build_node_template(node_name=topology_wiget_inputs['name'],node_type='tosca.nodes.QC.VM.topology',properties=topology_properties,requirements=topology_requirements))

# print(text)


topology_template={}
topology_template['node_templates'] = node_templates
tosca = {}
tosca['tosca_definitions_version'] = 'tosca_simple_yaml_1_2'
imports=[{'nodes':'https://raw.githubusercontent.com/qcdis-sdia/sdia-tosca/master/types/nodes.yaml','data':'https://raw.githubusercontent.com/qcdis-sdia/sdia-tosca/master/types/data.yml','capabilities':'https://raw.githubusercontent.com/qcdis-sdia/sdia-tosca/master/types/capabilities.yaml','policies':'https://raw.githubusercontent.com/qcdis-sdia/sdia-tosca/master/types/policies.yaml','interfaces':'https://raw.githubusercontent.com/qcdis-sdia/sdia-tosca/master/types/interfaces.yml'}]
tosca['imports'] = imports
repositories = {'docker_hub':'https://hub.docker.com/'}
tosca['repositories'] = repositories
tosca['topology_template'] = topology_template


## Show Topology

In [ ]:



def build_graph(node_templates):
    graph = nx.DiGraph()
    for node_name in node_templates:
        graph.add_node(node_name, attr_dict=node_templates[node_name])
        if 'requirements' in node_templates[node_name]:
            for req in node_templates[node_name]['requirements']:
                req_name = next(iter(req))
                req_node_name = req[req_name]['node']
                if 'relationship' in req[req_name] and 'type' in req[req_name]['relationship']:
                    relationship_type = req[req_name]['relationship']['type']
                else:
                    if 'relationship' not in req[req_name]:
                        relationship_type = 'tosca.relationships.DependsOn'
                    else:
                        relationship_type = req[req_name]['relationship']
                graph.add_edge(node_name, req_node_name, relationship=relationship_type)

    # nx.draw(graph, with_labels=True)
    # plt.savefig("/tmp/graph.png")
    # plt.show()
    return graph

def get_tosca(tosca_id):
    path = "/tosca_template/"+tosca_id
    payload = {}
    headers= {'accept':'text/plain'}
    response = requests.request("GET", base_url+path, headers=headers, data = payload,verify=False, auth=(username, password))
    return response.text

tosca = get_tosca(tosca_id)
tosca_dict = yaml.safe_load(tosca)
graph = build_graph(tosca_dict['topology_template']['node_templates'])
nx.draw(graph, with_labels=True)


## Make Plan

In [ ]:
path = "/planner/plan/"+tosca_id
payload = {}
headers= {'accept':'text/plain'}

response = requests.request("GET", base_url+path, headers=headers, data = payload,verify=False, auth=(username, password))
planed_tosca_id = response.text

In [ ]:
tosca = get_tosca(planed_tosca_id)

tosca_dict = yaml.safe_load(tosca)
graph = build_graph(tosca_dict['topology_template']['node_templates'])
nx.draw(graph, with_labels=True)

for node_name in tosca_dict['topology_template']['node_templates']:
    if 'properties' in tosca_dict['topology_template']['node_templates'][node_name]:
        print(node_name+': '+str(tosca_dict['topology_template']['node_templates'][node_name]['properties']))
        print('---------------------------------------------------------------------------------------------------------------------')
        
        

## Provision Plan

In [ ]:
path = "/provisioner/provision/"+planed_tosca_id
payload = {}
headers= {'accept':'text/plain'}

response = requests.request("GET", base_url+path, headers=headers, data = payload,verify=False, auth=(username, password))
provisioned_tosca_id = response.text

In [ ]:
tosca = get_tosca(provisioned_tosca_id)

tosca_dict = yaml.safe_load(tosca)
graph = build_graph(tosca_dict['topology_template']['node_templates'])
nx.draw(graph, with_labels=True)


for node_name in tosca_dict['topology_template']['node_templates']:
    if tosca_dict['topology_template']['node_templates'][node_name]['type'] == 'tosca.nodes.QC.VM.Compute':
        print(node_name+': '+tosca_dict['topology_template']['node_templates'][node_name]['attributes']['public_ip'])

## Deploy K8s And WS

In [ ]:
path = "/deployer/deploy/"+provisioned_tosca_id
payload = {}
headers= {'accept':'text/plain'}

response = requests.request("GET", base_url+path, headers=headers, data = payload, verify=False, auth=(username, password))
deployed_tosca_id = response.text

In [ ]:
tosca = get_tosca(deployed_tosca_id)

tosca_dict = yaml.safe_load(tosca)
graph = build_graph(tosca_dict['topology_template']['node_templates'])
nx.draw(graph, with_labels=True)

for node_name in tosca_dict['topology_template']['node_templates']:
    if tosca_dict['topology_template']['node_templates'][node_name]['type'] == 'tosca.nodes.QC.docker.Orchestrator.Kubernetes':
        for token in tosca_dict['topology_template']['node_templates'][node_name]['attributes']['tokens']:
            if token['user'] == 'dashboard_token':
                print(node_name+': '+' dashboard_token: '+token['token'])
        print(node_name+': '+str(tosca_dict['topology_template']['node_templates'][node_name]['attributes']['dashboard_url']))
    if tosca_dict['topology_template']['node_templates'][node_name]['type'] == 'tosca.nodes.QC.Container.Application.Docker':
        print(node_name+': '+str(tosca_dict['topology_template']['node_templates'][node_name]['attributes']['service_url']))
        